In [1]:
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")


In [2]:
from cargar_dataset import cargar_dataset
from config_datasets import config_datasets
from evaluacion import evaluar_sampler_holdout
from custom_samplers import PCSMOTEWrapper


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score
from pc_smote import PCSMOTE

def evaluar_pcsmote_grid_search(nombre_dataset, config, percentiles_densidad, percentiles_riesgo, criterios_pureza):
    print(f"📂 Cargando dataset: {nombre_dataset}")
    X, y, _ = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep", ","),
        header=config.get("header", None),
        binarizar=False,
        tipo=config.get("tipo", "tabular")
    )

    # Aplanado si es imagen
    if config.get("tipo") == "imagen":
        X = X.reshape((X.shape[0], -1)).astype(np.float32)

    # PCA
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    n_components = min(X_train.shape[1], 100)
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    resultados = []

    for pdens in percentiles_densidad:
        for priesgo in percentiles_riesgo:
            for criterio in criterios_pureza:
                print(f"🧪 Evaluando: {nombre_dataset} | Densidad: {pdens} | Riesgo: {priesgo} | Criterio: {criterio}")


                sampler = PCSMOTE(
                    random_state=42,
                    percentil_densidad=pdens,
                    percentil_dist=priesgo,
                    criterio_pureza=criterio,
                    modo_espacial='3d'
                )

                try:
                    if hasattr(sampler, "fit_resample_multiclass"):
                        X_res, y_res = sampler.fit_resample_multiclass(X_train_pca, y_train)
                    else:
                        X_res, y_res = sampler.fit_resample(X_train_pca, y_train)

                    clf = RandomForestClassifier(n_estimators=100, random_state=42)
                    clf.fit(X_res, y_res)
                    y_pred = clf.predict(X_test_pca)

                    f1 = f1_score(y_test, y_pred, average='weighted')
                    balanced_acc = balanced_accuracy_score(y_test, y_pred)
                    mcc = matthews_corrcoef(y_test, y_pred)
                    kappa = cohen_kappa_score(y_test, y_pred)

                    print(f"✅ F1-score: {f1:.4f} | Balanced Acc: {balanced_acc:.4f} | MCC: {mcc:.4f} | Kappa: {kappa:.4f}")

                    resultados.append({
                        'dataset': nombre_dataset,
                        'densidad': pdens,
                        'riesgo': priesgo,
                        'pureza': criterio,
                        'f1_score': f1,
                        'balanced_accuracy': balanced_acc,
                        'mcc': mcc,
                        'cohen_kappa': kappa
                    })

                except Exception as e:
                    print(f"⚠️ Error: {e}")
                    resultados.append({
                        'dataset': nombre_dataset,
                        'densidad': pdens,
                        'riesgo': priesgo,
                        'pureza': criterio,
                        'f1_score': None,
                        'error': str(e)
                    })

    df = pd.DataFrame(resultados)
    df = df.sort_values(by="f1_score", ascending=False)
    df.to_csv(f"../resultados/pcsmote_grid_{nombre_dataset}.csv", index=False)
    print(f"📁 Resultados guardados en: ../resultados/pcsmote_grid_{nombre_dataset}.csv")
    return df


In [4]:
from config_datasets import config_datasets

percentiles_densidad = [25, 50, 75]
percentiles_riesgo = [25, 50, 75]
criterios_pureza = ["entropia", "proporcion"]

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset == "eurosat":
        continue  # Ya procesado

    print(f"\n=== Ejecutando grid para {nombre_dataset} ===")
    evaluar_pcsmote_grid_search(nombre_dataset, config, percentiles_densidad, percentiles_riesgo, criterios_pureza)



=== Ejecutando grid para ecoli ===
📂 Cargando dataset: ecoli
🧪 Evaluando: ecoli | Densidad: 25 | Riesgo: 25 | Criterio: entropia
📌 Total muestras minoritarias: 61
📌 Total muestras mayoritarias: 207
📌 Total muestras minoritarias: 2
📌 Total muestras mayoritarias: 266
⚠️ Muy pocas muestras minoritarias (2). Se requieren al menos 6. Devolviendo dataset original.
📌 Total muestras minoritarias: 2
📌 Total muestras mayoritarias: 266
⚠️ Muy pocas muestras minoritarias (2). Se requieren al menos 6. Devolviendo dataset original.
📌 Total muestras minoritarias: 28
📌 Total muestras mayoritarias: 240
📌 Total muestras minoritarias: 16
📌 Total muestras mayoritarias: 252
📌 Total muestras minoritarias: 4
📌 Total muestras mayoritarias: 264
⚠️ Muy pocas muestras minoritarias (4). Se requieren al menos 6. Devolviendo dataset original.
📌 Total muestras minoritarias: 41
📌 Total muestras mayoritarias: 227
✅ F1-score: 0.8974 | Balanced Acc: 0.9318 | MCC: 0.8668 | Kappa: 0.8601
🧪 Evaluando: ecoli | Densidad: 25

In [6]:
import pandas as pd

datasets = ['ecoli', 'wdbc', 'glass', 'heart']
resumen = []

for d in datasets:
    df = pd.read_csv(f'../resultados/pcsmote_grid_{d}.csv')

    # Ignorar filas con errores o f1_score faltante
    df_valid = df.dropna(subset=["f1_score"])

    if df_valid.empty:
        print(f"⚠️ No hay resultados válidos para {d}")
        continue

    mejor = df_valid.sort_values(by="f1_score", ascending=False).iloc[0].copy()
    mejor['dataset'] = d
    resumen.append(mejor)

resumen_df = pd.DataFrame(resumen)

# Reordenar columnas si es necesario
cols = ['dataset', 'densidad', 'riesgo', 'pureza', 'f1_score']
for metric in ['balanced_accuracy', 'mcc', 'cohen_kappa']:
    if metric in resumen_df.columns:
        cols.append(metric)

resumen_df = resumen_df[cols]
resumen_df.to_csv("../resultados/resumen_mejores_pcsmote.csv", index=False)
print("✅ Resumen guardado en ../resultados/resumen_mejores_pcsmote.csv")


✅ Resumen guardado en ../resultados/resumen_mejores_pcsmote.csv
